In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
# Ayarlar
workspace_path = "/Users/ranakaya/Downloads/GoruntuVeriseti/"
destination_folder = workspace_path + "hard-fake-vs-real-faces/"
batch_size = 64  # bir eğitim döngüsüne kaç tane veri verileceğini belirler (aynı anda)
image_size = 224

In [11]:
# Veri yükleme ve ön işleme
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = ImageFolder(root=os.path.join(destination_folder, "train"), transform=transform)
validation_data = ImageFolder(root=os.path.join(destination_folder, "validation"), transform=transform)
test_data = ImageFolder(root=os.path.join(destination_folder, "test"), transform=transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [12]:
# Model oluşturma
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.base_model = torchvision.models.mobilenet_v3_large(pretrained=True)
        num_features = self.base_model.classifier[-1].in_features
        self.base_model.classifier[-1] = nn.Linear(num_features, 1)

    def forward(self, x):
        return torch.sigmoid(self.base_model(x))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomModel().to(device)

# Kayıp fonksiyonu ve optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


/opt/anaconda3/envs/torchimg/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/torchimg/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
# Eğitim
num_epochs = 20
best_accuracy = 0.0
early_stopping_counter = 0
train_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    # Eğitim aşaması
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)

    # Doğrulama aşaması
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.float().to(device)
            outputs = model(images)
            predicted = torch.round(outputs)
            total += labels.size(0)
            correct += (predicted == labels.unsqueeze(1)).sum().item()
    accuracy = correct / total
    val_accuracies.append(accuracy)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Validation Accuracy: {accuracy:.4f}")

    # Early stopping kontrolü
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= 5:
            print("Early stopping triggered.")
            break


KeyboardInterrupt: 